# Dealing with dates and years

In [933]:
import pandas as pd
from datetime import datetime
import re

df = pd.read_excel("GSAF5.xls")

## Normalize dates

- `Reported` was removed from the date column
- `Sept` is not an accepted `datetime` so it was replaced with `Sep`
- There was a typo in one record `Nox` instead of `Nov`

In [936]:
def normalize_date(date):
    if isinstance(date, datetime): return date
    date = str(date).strip().lower()
    date = date.replace("reported", "")
    date = date.replace("september", "sep")
    date = date.replace("sept", "sep")
    date = date.replace("nox", "nov")
    return date

## Format dates

### Defining regular expressions

- **Short dates:** 30-Oct-2024 or similar
- **Long dates:** 30-October-2024 or similar
- **Years**: 4 consecutive numbers

### Running regular expressions
- If the date is already a Python `datetime` object, return the date as it is
- First match long dates, short dates then years
- Parse these dates into a Python `datetime` object
- If there are any errors, return the date as it is

In [939]:
any = ".*?"
numbers = "[0-9]"
letters = "[a-zA-Z]"
separators = "[-\\s]"

short_date_regex = f"^{any}({numbers}{1,2}){separators}+({letters}{3,4}){separators}+({numbers}{4}){any}$"
long_date_regex = f"^{any}({numbers}{1,2}){separators}+({letters}{4,10}){separators}+({numbers}{4}){any}$"
year_regex = "^[0-9]{4}$"

def match_date(regex, date):
    match = re.search(regex, date)
    if match:
        day = long_match.group(1).zfill(2)
        month = long_match.group(2)
        year = long_match.group(3)
        return datetime.strptime(f"{day}/{month}/{year}", "%d/%B/%Y")
    return None

def format_date(date):
    if isinstance(date, datetime): return date

    try:
        long_date = match_date(long_date_regex, date)
        if long_date: return long_date

        short_date = match_date(short_date_regex, date)
        if short_date: return short_date

        year_match = re.search(year_regex, date)
        if year_match: return datetime.strptime(year_match.group(0), "%Y")
    except ValueError: return date

## Normalize years

- For years greater than or equal to 1000, return them as they are
- For dates between 100 and 999 (e.g. 950), add `1` to the beginning (e.g. 1950)
- For dates between 25 and 99 (e.g. 93), add `19` to the beginning (e.g. 1993)
- For dates between 10 and 24 (e.g. 24), add `20` to the beginning (e.g. 2024)
- For dates between 0 and 9 (e.g. 4), add `200` to the beginning (e.g. 2004)

In [942]:
def normalize_year(year):
    if year >= 1000: return year
    if year >= 100: return float(f"1{year}")
    if year >= 25: return float(f"19{year}")
    if year >= 10: return float(f"20{year}")
    if year >= 0: return float(f"200{year}")
    return year

## Infer years from dates

If a year is missing and the date record is present, use the year part from the date column

In [945]:
def infer_year(row):
    if pd.isnull(row.Year): row.Year = row.Date.year
    return row

## Apply functions to DataFrame

1. Normalize date
2. Format date
3. Drop empty dates otherwise
4. Normalize year
5. Infer year

In [948]:
df_copy = df.copy()

df_copy.Date = df_copy.Date.apply(normalize_date)
df_copy.Date = df_copy.Date.apply(format_date)
df_copy = df_copy.dropna(subset="Date")

df_copy.Year = df_copy.Year.apply(normalize_year)
df_copy[["Date", "Year"]] = df_copy[["Date", "Year"]].apply(infer_year, axis=1)

df_copy.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2024-10-14 00:00:00,2024.0,Unprovoked,Honduras,Tela,Atlantida,Swimming,Roberto Carlose Bu Mejia,M,38,...,Bull shark,Georgienne Bradley: Daily Mail UK: FTN News.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-11 00:00:00,2024.0,Unprovoked,USA,Florida,Brevard County Orlando,Surfing,Teddy Witteman,M,16,...,Bull shark 6ft,Todd SmithFlorida today: News 4:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-09-17 00:00:00,2024.0,Unprovoked,Honduras,Trujillo Colon,Puerto Castillo,Diving for shellfish,Fernando Mendoza Ocampo,M,33,...,Bull shark 6-7ft,Daily Mail UK: The Sun UK: The World Watch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-09-16 00:00:00,2024.0,Unprovoked,Morocco,Southern Morocco,West of Dakhla,Swimming - jumped off yacht,German Tourist,F,30,...,Reportedly a Great White,Andy Currie: Moroccan World News: Sky News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-08-26 00:00:00,2024.0,Unprovoked,Jamaica,Montego Bay,Falmouth,Spearfishing,Jahmari Reid,M,16,...,Reportedly Tiger Shark,Todd Smith: Daily Mail UK: Sky News: People .com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
